In [3]:
import pandas as pd

# 读取数据文件
file_path = '/mnt/d/Research/PHD/DLEPS/data/compoundinfo_beta.txt'
data = pd.read_csv(file_path, sep='\t')

# 统计总行数
total_rows = data.shape[0]

# 统计canonical_smiles列中不重复数据的数量
unique_smiles = data['canonical_smiles'].nunique()

# 统计每组canonical_smiles的条数
smiles_counts = data['canonical_smiles'].value_counts()

# 输出结果
print(f"总行数: {total_rows}")
print(f"canonical_smiles列中不重复数据的数量: {unique_smiles}")
print(f"每组数据的条数:")
print(smiles_counts)

总行数: 39321
canonical_smiles列中不重复数据的数量: 28575
每组数据的条数:
CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(c3)C(F)(F)F)c(F)c2)ccn1                  96
CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2/C(=O)Nc3ccc(F)cc23)c1C                        80
CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(c3)C(F)(F)F)cc2)ccn1                     66
CNC(=O)c1cccc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(c3)C(F)(F)F)cc2)c1                     66
CC1(C)O[C@@H]2CO[C@@]3(COS(N)(=O)=O)OC(C)(C)O[C@H]3[C@@H]2O1                  66
                                                                              ..
C[C@@H](CO)N1C[C@H](C)[C@H](CN(C)C(=O)Cc2ccccc2)OCc3cn(CCCC1=O)nn3             1
CO[C@H]1CN(C)C(=O)c2ccc(NC(=O)C3CCCC3)cc2OC[C@@H](C)N(C[C@@H]1C)S(C)(=O)=O     1
OC[C@@H]1O[C@H](CC(=O)NCC2CC2)C[C@@H]3[C@H]1Oc4ccc(NC(=O)CC5CC5)cc34           1
COc1cccc(c1)c2nc(no2)c3ccc(cc3)C(=O)O                                          1
OC(=O)c1ccccc1                                                                 1
Name: canonical_smiles, Length: 28575, dtype: int64


# 从`LINCS_small_molecules.tsv`中根据`GSE92742_Broad_LINCS_sig_info.txt`里面的BRD标识符匹配化合物SMILES
## 步骤一：准备数据

1. **加载数据**：

   - 从`GSE92742_Broad_LINCS_sig_info.txt`文件加载信号信息数据到`sig_info_df`。
   - 从`LINCS_small_molecules.tsv`文件加载小分子数据到`small_molecules_df`。

## 步骤二：匹配SMILES

1. **合并数据**：
   - 使用`pert_id`作为键，将`sig_info_df`和`small_molecules_df`中的数据合并，保留所有`sig_info_df`中的行，以及`small_molecules_df`中匹配的`canonical_smiles`列。
2. **保存结果**：
   - 将合并后的数据保存到`GSE92742_Broad_LINCS_sig_info_with_smiles.txt`文件中。

## 步骤三：筛选包含BRD的行

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_with_smiles.txt`文件加载已添加SMILES的数据到`filtered_df`。
2. **筛选数据**：
   - 筛选出`pert_id`中包含"BRD"的行。
3. **保存结果**：
   - 将筛选后的数据保存到`GSE92742_Broad_LINCS_sig_info_filtered.txt`文件中。

## 步骤四：筛选有5次以上重复的分子

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_filtered.txt`文件加载筛选后的数据到`replicated_df`。
2. **统计SMILES出现次数**：
   - 统计每个`canonical_smiles`的出现次数。
   - 筛选出出现次数大于5次的SMILES。
3. **筛选数据**：
   - 保留SMILES出现次数大于5次的行。
4. **保存结果**：
   - 将筛选后的数据保存到`GSE92742_Broad_LINCS_sig_info_replicated.txt`文件中。

## 步骤五：验证SMILES的有效性

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_replicated.txt`文件加载数据到`validated_df`。
2. **验证SMILES**：
   - 使用RDKit库验证每个SMILES是否有效。
3. **筛选数据**：
   - 保留SMILES有效的行。
4. **保存结果**：
   - 将验证后的数据保存到`GSE92742_Broad_LINCS_sig_info_final.txt`文件中。

## 步骤六：统计基本信息

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_final.txt`文件加载最终处理后的数据到`sig_info`。
2. **统计信息**：
   - 输出前五行数据。
   - 统计唯一`sig_id`、`pert_id`和`pert_iname`的数量。
   - 统计包含"BRD"的独一无二的`pert_id`数量。


In [4]:
import pandas as pd
from rdkit import Chem

# 文件路径配置
sig_info_path = '/mnt/d/Research/PHD/DLEPS/data/GSE92742/GSE92742_Broad_LINCS_sig_info.txt'
small_molecules_path = '/mnt/d/Research/PHD/DLEPS/data/GSE92742/LINCS_small_molecules.tsv'
output_with_smiles_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt'
output_filtered_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_filtered.txt'
output_replicated_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_replicated.txt'
output_final_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt'

In [5]:
def analyze_statistics(file_path):
    """统计文件中的基本信息"""
    df = pd.read_csv(file_path, sep="\t", dtype=str)
    
    # 总数量统计
    total_sig_id_count = df['sig_id'].count()
    total_pert_id_count = df['pert_id'].count()
    total_pert_iname_count = df['pert_iname'].count()
    total_canonical_smiles_count = df['canonical_smiles'].count()
    
    # 不重复数量统计
    unique_sig_id_count = df['sig_id'].nunique()
    unique_pert_id_count = df['pert_id'].nunique()
    unique_pert_iname_count = df['pert_iname'].nunique()
    unique_canonical_smiles_count = df['canonical_smiles'].nunique()
    
    # canonical_smiles为空和不为空的统计
    empty_smiles_count = df['canonical_smiles'].isna().sum()
    non_empty_smiles_count = df['canonical_smiles'].notna().sum()
    
    # pert_id 开头是 BRD 的统计
    brd_entries = df[df['pert_id'].str.startswith('BRD', na=False)]
    total_brd_count = brd_entries.shape[0]
    unique_brd_pert_id_count = brd_entries['pert_id'].nunique()
    empty_smiles_in_brd_count = brd_entries['canonical_smiles'].isna().sum()
    non_empty_smiles_in_brd_count = brd_entries['canonical_smiles'].notna().sum()
    
    # BRD 开头的条目中重复次数最多的前五个 pert_id
    top_brd_pert_ids = (
        brd_entries['pert_id']
        .value_counts()
        .head(5)
    )
    
    # pert_type 列的统计
    pert_type_counts = df['pert_type'].value_counts()
    total_pert_type_count = len(pert_type_counts)
    
    # 查找 canonical_smiles 相同但 pert_id 不同的数据
    duplicated_smiles = df[df.duplicated(subset=['canonical_smiles'], keep=False)]
    result = []
    for smile, group in duplicated_smiles.groupby('canonical_smiles'):
        if group['pert_id'].nunique() > 1:
            result.append(group)
    
    if result:
        result_df = pd.concat(result)
        result_file_path = '/mnt/d/Research/PHD/DLEPS/results/duplicated_smiles_diff_pertid.csv'
        result_df.to_csv(result_file_path, index=False)
        duplicated_smiles_message = f"数据已保存到文件: {result_file_path}"
    else:
        duplicated_smiles_message = "没有找到具有相同 canonical_smiles 但不同 pert_id 的数据。"

    # 打印统计信息
    print(f"文件: {file_path}")
    print(f"总数量统计:")
    print(f"  sig_id 总数: {total_sig_id_count}")
    print(f"  pert_id 总数: {total_pert_id_count}")
    print(f"  pert_iname 总数: {total_pert_iname_count}")
    print(f"  canonical_smiles 总数: {total_canonical_smiles_count}")
    print(f"不重复数量统计:")
    print(f"  不重复的 sig_id 数量: {unique_sig_id_count}")
    print(f"  不重复的 pert_id 数量: {unique_pert_id_count}")
    print(f"  不重复的 pert_iname 数量: {unique_pert_iname_count}")
    print(f"  不重复的 canonical_smiles 数量: {unique_canonical_smiles_count}")
    print(f"附加统计信息:")
    print(f"  具有空 canonical_smiles 的 pert_id 数量: {empty_smiles_count}")
    print(f"  具有非空 canonical_smiles 的 pert_id 数量: {non_empty_smiles_count}")
    print(f"pert_id 开头是 BRD 的统计:")
    print(f"  BRD 开头的条目总数: {total_brd_count}")
    print(f"  不重复的 BRD 开头的 pert_id 数量: {unique_brd_pert_id_count}")
    print(f"  BRD 开头的条目中，canonical_smiles 为空的数量: {empty_smiles_in_brd_count}")
    print(f"  BRD 开头的条目中，canonical_smiles 不为空的数量: {non_empty_smiles_in_brd_count}")
    print(f"  BRD 开头的条目中重复次数最多的前五个 pert_id 名称及其重复次数:")
    print(top_brd_pert_ids)
    print(f"pert_type 列的统计:")
    print(f"  不同种类的数量: {total_pert_type_count}")
    print(f"  每个种类的数量:")
    print(pert_type_counts)
    print(f"相同 canonical_smiles 但不同 pert_id 的数据检查:")
    print(duplicated_smiles_message)
    print("-" * 50)


In [6]:
def add_smiles_column():
    """Step 1-3: 匹配 SMILES 并添加到数据"""
    sig_info_df = pd.read_csv(sig_info_path, sep='\t')
    small_molecules_df = pd.read_csv(small_molecules_path, sep='\t')
    merged_df = sig_info_df.merge(small_molecules_df[['pert_id', 'canonical_smiles']], on='pert_id', how='left')
    merged_df.to_csv(output_with_smiles_path, sep='\t', index=False)
    print(f"添加canonical_smiles列后的结果已保存到: {output_with_smiles_path}")
    analyze_statistics(output_with_smiles_path)  # 输出文件统计信息

add_smiles_column()


/home/cpu/anaconda3/envs/DLEPS/lib/python3.7/site-packages/ipykernel_launcher.py:10: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.


添加canonical_smiles列后的结果已保存到: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt
文件: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt
总数量统计:
  sig_id 总数: 473647
  pert_id 总数: 473647
  pert_iname 总数: 473647
  canonical_smiles 总数: 204986
不重复数量统计:
  不重复的 sig_id 数量: 473647
  不重复的 pert_id 数量: 51219
  不重复的 pert_iname 数量: 28927
  不重复的 canonical_smiles 数量: 20314
附加统计信息:
  具有空 canonical_smiles 的 pert_id 数量: 268661
  具有非空 canonical_smiles 的 pert_id 数量: 204986
pert_id 开头是 BRD 的统计:
  BRD 开头的条目总数: 204933
  不重复的 BRD 开头的 pert_id 数量: 20402
  BRD 开头的条目中，canonical_smiles 为空的数量: 48
  BRD 开头的条目中，canonical_smiles 不为空的数量: 204885
  BRD 开头的条目中重复次数最多的前五个 pert_id 名称及其重复次数:
BRD-K81418486    1323
BRD-A75409952     962
BRD-A19500257     928
BRD-A19037878     858
BRD-K27305650     375
Name: pert_id, dtype: int64
pert_type 列的统计:
  不同种类的数量: 13
  每个种类的数量:
trt_cp             205034
trt_sh             154993
trt_sh.cgs          36720
trt_sh.css          24368
tr

In [7]:
def filter_brd_data():
    """Step 4-5: 筛选出包含 BRD 的行"""
    merged_df = pd.read_csv(output_with_smiles_path, sep='\t')
    filtered_df = merged_df[merged_df['pert_id'].str.contains('BRD', na=False)]
    filtered_df.to_csv(output_filtered_path, sep='\t', index=False)
    print(f"过滤后的结果（仅保留pert_id包含BRD）已保存到: {output_filtered_path}")
    analyze_statistics(output_filtered_path)  # 输出文件统计信息

filter_brd_data()


/home/cpu/anaconda3/envs/DLEPS/lib/python3.7/site-packages/ipykernel_launcher.py:9: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  if __name__ == "__main__":


过滤后的结果（仅保留pert_id包含BRD）已保存到: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_filtered.txt
文件: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_filtered.txt
总数量统计:
  sig_id 总数: 204934
  pert_id 总数: 204934
  pert_iname 总数: 204934
  canonical_smiles 总数: 204885
不重复数量统计:
  不重复的 sig_id 数量: 204934
  不重复的 pert_id 数量: 20403
  不重复的 pert_iname 数量: 19802
  不重复的 canonical_smiles 数量: 20314
附加统计信息:
  具有空 canonical_smiles 的 pert_id 数量: 49
  具有非空 canonical_smiles 的 pert_id 数量: 204885
pert_id 开头是 BRD 的统计:
  BRD 开头的条目总数: 204933
  不重复的 BRD 开头的 pert_id 数量: 20402
  BRD 开头的条目中，canonical_smiles 为空的数量: 48
  BRD 开头的条目中，canonical_smiles 不为空的数量: 204885
  BRD 开头的条目中重复次数最多的前五个 pert_id 名称及其重复次数:
BRD-K81418486    1323
BRD-A75409952     962
BRD-A19500257     928
BRD-A19037878     858
BRD-K27305650     375
Name: pert_id, dtype: int64
pert_type 列的统计:
  不同种类的数量: 2
  每个种类的数量:
trt_cp    204933
trt_oe         1
Name: pert_type, dtype: int64
相同 canonical_smiles 但不同 pert_id 的数据检查:
数据已保存到文件: /mnt

In [8]:
def filter_replicates():
    """Step 6: 保留有5次以上重复的分子"""
    filtered_df = pd.read_csv(output_filtered_path, sep='\t')
    
    # 统计每个 SMILES 的出现次数
    smiles_counts = filtered_df['canonical_smiles'].value_counts()
    replicated_smiles = smiles_counts[smiles_counts > 5].index
    replicated_df = filtered_df[filtered_df['canonical_smiles'].isin(replicated_smiles)]
    
    replicated_df.to_csv(output_replicated_path, sep='\t', index=False)
    print(f"筛选后（SMILES 出现次数大于 5）的数据已保存到: {output_replicated_path}")
    analyze_statistics(output_replicated_path)  # 输出文件统计信息

filter_replicates()


筛选后（SMILES 出现次数大于 5）的数据已保存到: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_replicated.txt
文件: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_replicated.txt
总数量统计:
  sig_id 总数: 169336
  pert_id 总数: 169336
  pert_iname 总数: 169336
  canonical_smiles 总数: 169336
不重复数量统计:
  不重复的 sig_id 数量: 169336
  不重复的 pert_id 数量: 8978
  不重复的 pert_iname 数量: 8563
  不重复的 canonical_smiles 数量: 8891
附加统计信息:
  具有空 canonical_smiles 的 pert_id 数量: 0
  具有非空 canonical_smiles 的 pert_id 数量: 169336
pert_id 开头是 BRD 的统计:
  BRD 开头的条目总数: 169336
  不重复的 BRD 开头的 pert_id 数量: 8978
  BRD 开头的条目中，canonical_smiles 为空的数量: 0
  BRD 开头的条目中，canonical_smiles 不为空的数量: 169336
  BRD 开头的条目中重复次数最多的前五个 pert_id 名称及其重复次数:
BRD-K81418486    1323
BRD-A75409952     962
BRD-A19500257     928
BRD-A19037878     858
BRD-K27305650     375
Name: pert_id, dtype: int64
pert_type 列的统计:
  不同种类的数量: 1
  每个种类的数量:
trt_cp    169336
Name: pert_type, dtype: int64
相同 canonical_smiles 但不同 pert_id 的数据检查:
数据已保存到文件: /mnt/d/Research/PHD/DLE

In [9]:
def validate_smiles():
    """Step 7-8: 验证 SMILES 是否有效"""
    replicated_df = pd.read_csv(output_replicated_path, sep='\t')
    
    # 验证 SMILES 是否能被 RDKit 解析
    def is_valid_smiles(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
            return mol is not None
        except:
            return False
    
    validated_df = replicated_df[replicated_df['canonical_smiles'].apply(is_valid_smiles)]
    validated_df.to_csv(output_final_path, sep='\t', index=False)
    print(f"最终处理后的数据已保存到: {output_final_path}")
    analyze_statistics(output_final_path)  # 输出文件统计信息

validate_smiles()


[11:00:42] SMILES Parse Error: syntax error while parsing: -
[11:00:42] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:00:42] SMILES Parse Error: syntax error while parsing: -
[11:00:42] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:00:42] SMILES Parse Error: syntax error while parsing: -
[11:00:42] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:00:42] SMILES Parse Error: syntax error while parsing: -
[11:00:42] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:00:42] SMILES Parse Error: syntax error while parsing: -
[11:00:42] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:00:42] SMILES Parse Error: syntax error while parsing: -
[11:00:42] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:00:42] SMILES Parse Error: syntax error while parsing: -
[11:00:42] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:00:42] SMILES Parse Error: syntax error while parsing: -
[11:00:4

最终处理后的数据已保存到: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt
文件: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt
总数量统计:
  sig_id 总数: 167430
  pert_id 总数: 167430
  pert_iname 总数: 167430
  canonical_smiles 总数: 167430
不重复数量统计:
  不重复的 sig_id 数量: 167430
  不重复的 pert_id 数量: 8914
  不重复的 pert_iname 数量: 8525
  不重复的 canonical_smiles 数量: 8889
附加统计信息:
  具有空 canonical_smiles 的 pert_id 数量: 0
  具有非空 canonical_smiles 的 pert_id 数量: 167430
pert_id 开头是 BRD 的统计:
  BRD 开头的条目总数: 167430
  不重复的 BRD 开头的 pert_id 数量: 8914
  BRD 开头的条目中，canonical_smiles 为空的数量: 0
  BRD 开头的条目中，canonical_smiles 不为空的数量: 167430
  BRD 开头的条目中重复次数最多的前五个 pert_id 名称及其重复次数:
BRD-K81418486    1323
BRD-A75409952     962
BRD-A19500257     928
BRD-A19037878     858
BRD-K27305650     375
Name: pert_id, dtype: int64
pert_type 列的统计:
  不同种类的数量: 1
  每个种类的数量:
trt_cp    167430
Name: pert_type, dtype: int64
相同 canonical_smiles 但不同 pert_id 的数据检查:
数据已保存到文件: /mnt/d/Research/PHD/DLEPS/results/duplicated_smi